In [263]:
import pandas as pd
import numpy as np
#este se usara para guardar fechas con la semana actual
import datetime

#esto se usaran para enviar emails
from dotenv import dotenv_values
from email.message import EmailMessage
import ssl
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
#import os


DATASETS

In [264]:
#leemos los datasets
deliveries = pd.read_csv('data/deliveries.csv')
shifts = pd.read_csv('data/shifts.csv')
performance = pd.read_csv('data/performance2.csv')
justificaciones = pd.read_csv('data/justificaciones.csv')

In [265]:
deliveries

,driver_id,delivery_id,zone_name,date,del_cancelation_reason,del_accepted_at,del_canceled_at
0,276993,307323718,Madrid,2024-01-22,Late / Unresponsive,2024-01-22 09:04:55.71,2024-01-22 09:15:52.276
1,276993,308875307,Madrid,2024-02-05,Late / Unresponsive,2024-02-05 09:31:36.08,2024-02-05 09:48:14.719
2,280920,306137597,Madrid,2024-01-08,Late / Unresponsive,2024-01-08 08:37:25.046,2024-01-08 08:47:47.861
3,280920,306137598,Madrid,2024-01-08,Late / Unresponsive,2024-01-08 08:37:25.046,2024-01-08 08:47:47.877
4,280920,306692552,Madrid,2024-01-13,Late / Unresponsive,2024-01-13 09:35:07.657,2024-01-13 09:46:16.642
...,...,...,...,...,...,...,...
1417,688800,309112406,Valles,2024-02-02,Late / Unresponsive,2024-02-02 23:10:34.702,2024-02-02 23:28:20.165
1418,688800,309114781,Valles,2024-02-02,Late / Unresponsive,2024-02-02 23:35:52.32,2024-02-02 23:58:58.666
1419,694272,306693561,Barcelona,2024-01-12,Refused delivery,2024-01-12 22:41:49.214,2024-01-12 22:42:44.738
1420,694272,307505381,Barcelona,2024-01-19,Refused delivery,2024-01-19 22:52:33.509,2024-01-19 23:08:20.727


In [266]:
shifts

,driver_id,date,shift_duration,abscence
0,144250,2024-02-01,3.0,1.000000
1,144250,2024-02-05,6.0,1.000000
2,144250,2024-02-20,6.0,1.000000
3,171353,2024-01-21,7.0,1.000000
4,171353,2024-01-24,7.0,1.000000
...,...,...,...,...
449,685296,2024-02-17,3.0,1.000000
450,685296,2024-02-18,3.0,1.000000
451,685296,2024-02-19,3.0,1.000000
452,685296,2024-02-20,3.0,1.000000


In [267]:
performance


,driver_id,week,transport_type,tenure,deliveries_invoiced,e2e_invoiced,client_price_invoiced,driver_earnings_invoiced,deliveries_not_invoiced,e2e_not_invoiced,...,no_disponible,assigned_shifts,is_in_area_begin,no_shows,invitations,non_commit,lack_equipment,late_unresponsive,refused_delivery,equipment_failure
0,144250,2.0,e_motorbikexl,NaN,10,17980,59.50,3.32,0,0,...,0.0,4,4,0,8.0,0.0,NaN,NaN,NaN,NaN
1,144250,3.0,e_motorbikexl,NaN,8,14153,49.99,2.83,0,0,...,0.0,8,6,2,7.0,0.0,NaN,NaN,NaN,NaN
2,144250,4.0,e_motorbikexl,NaN,10,12647,55.25,1.95,0,0,...,0.0,8,8,0,8.0,0.0,NaN,NaN,NaN,NaN
3,144250,5.0,e_motorbikexl,NaN,10,18030,62.74,3.26,0,0,...,0.0,10,9,1,10.0,0.0,NaN,NaN,NaN,NaN
4,144250,6.0,e_motorbikexl,NaN,11,15202,64.07,2.36,0,0,...,0.0,10,8,2,10.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079,694272,6.0,e_bike,37.0,17,19423,96.02,0.00,0,0,...,0.0,5,5,0,17.0,0.0,NaN,NaN,NaN,NaN
1080,694272,7.0,e_bike,37.0,16,22406,93.12,0.00,2,581,...,0.0,6,6,0,17.0,1.0,NaN,NaN,NaN,1.0
1081,694272,8.0,e_bike,37.0,16,18992,91.11,0.00,0,0,...,0.0,5,5,0,16.0,0.0,NaN,NaN,NaN,NaN
1082,694272,9.0,e_bike,37.0,13,18158,63.40,0.00,1,189,...,0.0,4,4,0,14.0,1.0,NaN,NaN,1.0,NaN


In [268]:
justificaciones

,Timestamp,ID Pedido,Justificado,Como se ha justificado
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,2/7/2024 12:27:05,302559894,NO,NaN
3,2/7/2024 12:44:21,302642374,NO,NaN
4,2/7/2024 12:48:22,171353,NO,NaN
...,...,...,...,...
541,3/6/2024 14:29:16,312042195,NO,NaN
542,3/6/2024 14:39:35,312374704,NO,NaN
543,3/6/2024 14:50:33,312237256,SI,Fin de turno.
544,3/7/2024 13:37:31,312732737,SI,Fallo de equipo.


# EDA

In [269]:
#Sacamos las medias frente a sus pedidos
performance['E2E Invoiced'] = performance['e2e_invoiced']/performance['deliveries_invoiced']/60
performance['E2E Succeeded'] = performance['e2e_succeeded']/performance['deliveries_succeeded']/60
performance['PACE Succeded'] = (performance['e2e_succeeded']/60)/((performance['distance_to_pu_succeeded']+performance['distance_to_do_succeeded'])/1000)
performance['WAPU'] = performance['wapu']/performance['performance_deliveries']/60
performance['WADO'] = performance['wado']/performance['performance_deliveries']/60
performance['PACE PU Succeded'] = (performance['picking_time'] / 
                                   (performance['picking_time'] + 
                                    performance['wapu'] + 
                                    performance['delivering_time'] + 
                                    performance['wado'])) * ((performance['e2e_succeeded'] / 
                                                              performance['deliveries_succeeded']) / 60) / ((performance['distance_to_pu_succeeded'] / 
                                                                                                             1000) / performance['deliveries_succeeded'])
performance['PACE DO Succeded'] = (performance['delivering_time'] / 
                                   (performance['picking_time'] + performance['wapu'] + 
                                    performance['delivering_time'] + 
                                    performance['wado'])) * ((performance['e2e_succeeded'] / 
                                                              performance['deliveries_succeeded']) / 60) / ((performance['distance_to_do_succeeded'] / 
                                                                                                             1000) / performance['deliveries_succeeded'])
performance['PACE PU vs DO'] = performance['PACE PU Succeded'] - performance['PACE DO Succeded']
performance['Connection Rate'] = performance['online_hours'] / performance['assigned_hours']
performance['Deliveries'] = performance['deliveries_invoiced']
performance['Online'] = performance['online_hours']
performance['Online Rate'] = np.where(performance['assigned_hours'] != 0,
                                performance['Online'] / performance['assigned_hours'],
                                0)
performance['Del/h'] = np.where(performance['Online'] != 0, performance['Deliveries'] / performance['Online'], 0)
performance['Cancelations'] = performance['non_commit'] / performance['invitations']
performance['IAB'] = performance['is_in_area_begin'] / performance['assigned_shifts']

In [270]:
#vemos como estan los datos
performance

,driver_id,week,transport_type,tenure,deliveries_invoiced,e2e_invoiced,client_price_invoiced,driver_earnings_invoiced,deliveries_not_invoiced,e2e_not_invoiced,...,PACE PU Succeded,PACE DO Succeded,PACE PU vs DO,Connection Rate,Deliveries,Online,Online Rate,Del/h,Cancelations,IAB
0,144250,2.0,e_motorbikexl,NaN,10,17980,59.50,3.32,0,0,...,5.923632,3.714918,2.208715,0.978333,10,11.74,0.978333,0.851789,0.000000,1.00
1,144250,3.0,e_motorbikexl,NaN,8,14153,49.99,2.83,0,0,...,5.905910,3.802057,2.103853,0.750000,8,18.00,0.750000,0.444444,0.000000,0.75
2,144250,4.0,e_motorbikexl,NaN,10,12647,55.25,1.95,0,0,...,6.205363,3.216219,2.989144,1.000000,10,24.00,1.000000,0.416667,0.000000,1.00
3,144250,5.0,e_motorbikexl,NaN,10,18030,62.74,3.26,0,0,...,6.965629,3.996889,2.968740,0.897667,10,26.93,0.897667,0.371333,0.000000,0.90
4,144250,6.0,e_motorbikexl,NaN,11,15202,64.07,2.36,0,0,...,6.490947,4.662328,1.828619,0.800000,11,24.00,0.800000,0.458333,0.000000,0.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079,694272,6.0,e_bike,37.0,17,19423,96.02,0.00,0,0,...,4.698284,4.882717,-0.184433,0.796000,17,11.94,0.796000,1.423786,0.000000,1.00
1080,694272,7.0,e_bike,37.0,16,22406,93.12,0.00,2,581,...,5.221761,5.302044,-0.080283,0.844444,16,15.20,0.844444,1.052632,0.058824,1.00
1081,694272,8.0,e_bike,37.0,16,18992,91.11,0.00,0,0,...,7.433031,5.814040,1.618992,0.769333,16,11.54,0.769333,1.386482,0.000000,1.00
1082,694272,9.0,e_bike,37.0,13,18158,63.40,0.00,1,189,...,6.730794,6.120836,0.609959,0.721667,13,8.66,0.721667,1.501155,0.071429,1.00


In [271]:
#Seleccionamos las columnas que nos interesan
columnas = ['driver_id','week','transport_type','E2E Invoiced','E2E Succeeded','PACE Succeded','WAPU','WADO','PACE PU Succeded','PACE DO Succeded','PACE PU vs DO','Connection Rate','Deliveries','Online','Online Rate','Del/h','Cancelations','IAB']

In [272]:
#Cremaos un nuevo df con las columnas que nos interesan.
transformed_performance = performance.loc[:, columnas]

In [273]:
transformed_performance

,driver_id,week,transport_type,E2E Invoiced,E2E Succeeded,PACE Succeded,WAPU,WADO,PACE PU Succeded,PACE DO Succeded,PACE PU vs DO,Connection Rate,Deliveries,Online,Online Rate,Del/h,Cancelations,IAB
0,144250,2.0,e_motorbikexl,29.966667,30.691667,6.680634,4.743750,4.758333,5.923632,3.714918,2.208715,0.978333,10,11.74,0.978333,0.851789,0.000000,1.00
1,144250,3.0,e_motorbikexl,29.485417,29.485417,7.063433,4.420833,4.406250,5.905910,3.802057,2.103853,0.750000,8,18.00,0.750000,0.444444,0.000000,0.75
2,144250,4.0,e_motorbikexl,21.078333,20.289583,7.763747,4.539583,3.895833,6.205363,3.216219,2.989144,1.000000,10,24.00,1.000000,0.416667,0.000000,1.00
3,144250,5.0,e_motorbikexl,30.050000,29.768519,7.544823,2.075000,6.470833,6.965629,3.996889,2.968740,0.897667,10,26.93,0.897667,0.371333,0.000000,0.90
4,144250,6.0,e_motorbikexl,23.033333,23.637037,8.274664,3.598148,4.057407,6.490947,4.662328,1.828619,0.800000,11,24.00,0.800000,0.458333,0.000000,0.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079,694272,6.0,e_bike,19.042157,19.042157,6.833214,2.962745,2.866667,4.698284,4.882717,-0.184433,0.796000,17,11.94,0.796000,1.423786,0.000000,1.00
1080,694272,7.0,e_bike,23.339583,23.339583,7.706171,2.881250,4.519792,5.221761,5.302044,-0.080283,0.844444,16,15.20,0.844444,1.052632,0.058824,1.00
1081,694272,8.0,e_bike,19.783333,19.783333,8.554955,1.985556,2.907778,7.433031,5.814040,1.618992,0.769333,16,11.54,0.769333,1.386482,0.000000,1.00
1082,694272,9.0,e_bike,23.279487,23.452778,8.850661,3.525000,2.723333,6.730794,6.120836,0.609959,0.721667,13,8.66,0.721667,1.501155,0.071429,1.00


In [274]:
#Seleccionamos las columnas númericas que para poder hacer un medía y agruparlas por driver_id
columnas_numericas = ['WAPU','WADO','PACE PU Succeded','PACE DO Succeded','PACE PU vs DO','Online Rate','Cancelations','IAB']

In [275]:
#Hacemos el group by y indicamos que nos haga las medías de las columnas númericas.
new_df = transformed_performance.groupby(['driver_id','transport_type'])[columnas_numericas].mean().reset_index()

In [276]:
#Miramos como es el nuevo df
new_df

,driver_id,transport_type,WAPU,WADO,PACE PU Succeded,PACE DO Succeded,PACE PU vs DO,Online Rate,Cancelations,IAB
0,144250,e_motorbikexl,3.697479,4.376769,6.399824,3.924967,2.474857,0.914000,0.000000,0.888889
1,276993,e_motorbikexl,5.185135,8.896818,6.150316,4.095716,2.054600,0.878657,0.018060,0.746296
2,280920,e_motorbikexl,4.836038,6.975193,4.503761,4.132391,0.371370,0.931866,0.035964,0.397917
3,283943,e_motorbikexl,4.846535,8.822913,5.049229,4.125518,0.923711,0.874069,0.027244,0.891667
4,284449,bike,5.650000,1.976667,5.351997,3.027322,2.324674,0.200000,0.000000,0.200000
...,...,...,...,...,...,...,...,...,...,...
142,683252,e_bike,7.121740,6.057356,7.038987,5.663577,1.375410,0.863639,0.009460,0.269444
143,685061,bike,2.997222,1.810648,5.269904,4.193455,1.076449,0.996444,0.000000,0.000000
144,685063,bike,5.325000,1.983333,28.841612,24.858553,3.983059,0.936000,0.500000,0.000000
145,688800,e_motorbike,2.918333,18.992500,8.312309,5.123986,3.188323,0.973955,0.285714,0.000000


In [277]:
#Normalizamos y sacamos nuevas columnas con el scooring de las columnas que hemos seleccionado
new_df['score_WAPU'] = (new_df['WAPU'] - new_df['WAPU'].min()) / (new_df['WAPU'].max() - new_df['WAPU'].min())
new_df['score_WADO'] = (new_df['WADO'] - new_df['WADO'].min()) / (new_df['WADO'].max() - new_df['WADO'].min())
new_df['score_PACE PU Succeded'] = (new_df['PACE PU Succeded'] - new_df['PACE PU Succeded'].min()) / (new_df['PACE PU Succeded'].max() - new_df['PACE PU Succeded'].min())
new_df['score_PACE DO Succeded'] = (new_df['PACE DO Succeded'] - new_df['PACE DO Succeded'].min()) / (new_df['PACE DO Succeded'].max() - new_df['PACE DO Succeded'].min())
new_df['score_PACE PU vs DO'] = (new_df['PACE PU vs DO'] - new_df['PACE PU vs DO'].min()) / (new_df['PACE PU vs DO'].max() - new_df['PACE PU vs DO'].min())
new_df['score_Online Rate'] = new_df['Online Rate']
new_df['score_Cancelations'] = new_df['Cancelations']
new_df['score_IAB'] = new_df['IAB']


In [278]:
#Miramos como ha quedado el nuevo df
new_df

,driver_id,transport_type,WAPU,WADO,PACE PU Succeded,PACE DO Succeded,PACE PU vs DO,Online Rate,Cancelations,IAB,score_WAPU,score_WADO,score_PACE PU Succeded,score_PACE DO Succeded,score_PACE PU vs DO,score_Online Rate,score_Cancelations,score_IAB
0,144250,e_motorbikexl,3.697479,4.376769,6.399824,3.924967,2.474857,0.914000,0.000000,0.888889,0.234173,0.052786,0.043802,0.041117,0.097660,0.914000,0.000000,0.888889
1,276993,e_motorbikexl,5.185135,8.896818,6.150316,4.095716,2.054600,0.878657,0.018060,0.746296,0.384711,0.135525,0.039977,0.048939,0.091302,0.878657,0.018060,0.746296
2,280920,e_motorbikexl,4.836038,6.975193,4.503761,4.132391,0.371370,0.931866,0.035964,0.397917,0.349385,0.100350,0.014737,0.050619,0.065838,0.931866,0.035964,0.397917
3,283943,e_motorbikexl,4.846535,8.822913,5.049229,4.125518,0.923711,0.874069,0.027244,0.891667,0.350448,0.134173,0.023099,0.050304,0.074194,0.874069,0.027244,0.891667
4,284449,bike,5.650000,1.976667,5.351997,3.027322,2.324674,0.200000,0.000000,0.200000,0.431752,0.008852,0.027740,0.000000,0.095388,0.200000,0.000000,0.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,683252,e_bike,7.121740,6.057356,7.038987,5.663577,1.375410,0.863639,0.009460,0.269444,0.580680,0.083549,0.053599,0.120756,0.081027,0.863639,0.009460,0.269444
143,685061,bike,2.997222,1.810648,5.269904,4.193455,1.076449,0.996444,0.000000,0.000000,0.163312,0.005814,0.026481,0.053416,0.076505,0.996444,0.000000,0.000000
144,685063,bike,5.325000,1.983333,28.841612,24.858553,3.983059,0.936000,0.500000,0.000000,0.398864,0.008975,0.387810,1.000000,0.120476,0.936000,0.500000,0.000000
145,688800,e_motorbike,2.918333,18.992500,8.312309,5.123986,3.188323,0.973955,0.285714,0.000000,0.155329,0.320326,0.073118,0.096040,0.108453,0.973955,0.285714,0.000000


In [279]:
#Sacamos un único scooring sumando las positivas y restando las que són negativas.
new_df['Total_Score'] = -new_df['score_WAPU']-new_df['score_WADO']-new_df['score_PACE PU Succeded']-new_df['score_PACE DO Succeded']-new_df['score_PACE PU vs DO']+new_df['score_Online Rate']-new_df['score_Cancelations']+new_df['score_IAB']

In [280]:
#Generamos el ranking para ver quienes son los tops.
final_df = new_df.sort_values(by='Total_Score',ascending=False).reset_index(drop=True)
print('RANKING GENERADO')

,driver_id,transport_type,WAPU,WADO,PACE PU Succeded,PACE DO Succeded,PACE PU vs DO,Online Rate,Cancelations,IAB,score_WAPU,score_WADO,score_PACE PU Succeded,score_PACE DO Succeded,score_PACE PU vs DO,score_Online Rate,score_Cancelations,score_IAB,Total_Score
0,427199,e_motorbike,3.609494,2.784293,3.981534,4.021644,-0.040110,0.997046,0.000000,0.936941,0.225269,0.023636,0.006732,0.045546,0.059613,0.997046,0.000000,0.936941,1.573191
1,370425,e_motorbike,2.280904,2.573067,4.493187,4.748133,-0.254946,0.965720,0.007538,0.859259,0.090827,0.019770,0.014575,0.078823,0.056363,0.965720,0.007538,0.859259,1.557083
2,653770,e_motorbike,3.264297,4.915756,4.373149,4.111342,0.261807,0.963806,0.006481,0.977778,0.190338,0.062652,0.012735,0.049655,0.064181,0.963806,0.006481,0.977778,1.555541
3,505787,motorbike,1.889048,3.471548,5.843615,5.006640,0.836975,0.989833,0.000000,0.850000,0.051174,0.036216,0.035276,0.090664,0.072882,0.989833,0.000000,0.850000,1.553621
4,476245,e_bike,3.761104,3.093548,5.493656,5.115362,0.378294,1.000000,0.002096,1.000000,0.240611,0.029297,0.029911,0.095645,0.065943,1.000000,0.002096,1.000000,1.536497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,405141,motorbike,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,1.000000,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,1.000000,NaN
143,512711,bike,NaN,NaN,NaN,NaN,NaN,0.762333,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,0.762333,1.000000,0.000000,NaN
144,619693,bike,NaN,NaN,NaN,NaN,NaN,0.552333,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,0.552333,1.000000,0.000000,NaN
145,650097,bike,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.400000,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.400000,NaN


In [281]:
#generamos csv para subirlo a tableau en caso de que queramos tratar esos datos
final_df.to_csv('backups/ranking6w.csv',index=True,index_label='rank')
print('csv generado')

csv generado


In [282]:
#creamos un df de los datos de los drivers para poder incluir sus datos en el mensaje y poder concatenar su correo electronico.
email_df = final_df.sort_values(by='Total_Score',ascending=False).reset_index(drop=True)
email_df = email_df.drop(['transport_type','score_WAPU','score_WADO','score_PACE PU Succeded','score_PACE DO Succeded','score_PACE PU vs DO','score_Online Rate','score_Cancelations','score_IAB'],axis=1)

In [283]:
#Eliminamos las columnas que no necesitamos para que el mensaje sea más facíl de hacer.
final_df = final_df.drop(['transport_type','WAPU','WADO','PACE PU Succeded','PACE DO Succeded','PACE PU vs DO','Online Rate','Cancelations','IAB'],axis=1)

In [284]:
# Calcular los valores mínimo y máximo de la columna 'score'
score_min = final_df['Total_Score'].min()
score_max = final_df['Total_Score'].max()

# Definir los porcentajes para los rangos(se puede cambiar, 
#...pero si queremos hacer más grupos habría que cambiar el códgio de la función)
porcentaje_superior = 0.9
porcentaje_medio = 0.7
porcentaje_bajo = 0.5

# Calcular los rangos en función de los porcentajes
rango_superior = score_min + (score_max - score_min) * porcentaje_superior
rango_medio = score_min + (score_max - score_min) * porcentaje_medio
rango_bajo = score_min + (score_max - score_min) * porcentaje_bajo

def enviar_mensaje(row):
    total_score = row['Total_Score']
    if not np.isnan(total_score):
        if total_score >= rango_superior:
            return "¡Excelente trabajo!"
        elif total_score >= rango_medio:
            return "Bien hecho."
        elif total_score >= rango_bajo:
            return "Sigue mejorando."
        else:
            # Obtener las columnas numéricas para encontrar el mínimo
            columnas_numericas = row.drop('Total_Score').loc[lambda x: pd.to_numeric(x, errors='coerce').notnull()]
            if not columnas_numericas.empty:
                columnas_trabajo = [key for key, value in columnas_numericas.items() if value == min(columnas_numericas.values)]
                mensajes = []
                # Ronda de IFs para que no quede feo el nombre de la columna
                if 'score_WAPU' in columnas_trabajo:
                    mensajes.append("¡El tiempo de espera en el local necesita mejorar!")
                if 'score_WADO' in columnas_trabajo:
                    mensajes.append("¡El cierre de los pedidos necesita mejorar!")
                if 'score_PACE PU Succeded' in columnas_trabajo:
                    mensajes.append("¡El ritmo de recogida necesita mejorar!")
                if 'score_PACE DO Succeded' in columnas_trabajo:
                    mensajes.append("¡El ritmo en la entrega necesita mejorar!")
                if 'score_PACE PU vs DO' in columnas_trabajo:
                    mensajes.append("Tardas más en ir a recoger que en entregar")
                if 'score_Online Rate' in columnas_trabajo:
                    mensajes.append("¡Tienes que asegurar estar conectado en tus horas!")
                if 'score_Cancelations' in columnas_trabajo:
                    mensajes.append("¡Recuerda que cancelar puede llevar a sanciones!")
                if 'score_IAB' in columnas_trabajo:
                    mensajes.append("¡Tienes que mejorar la puntualidad!")
                # En caso de que necesitemos más añadir aquí
                return " ".join(mensajes)
            else:
                return "No hay datos numéricos para comparar."
    else:
        return "No hay puntuación total disponible."

        
final_df['mensaje'] = final_df.apply(enviar_mensaje, axis=1)
print(f'Mensajes Generados')
print(final_df)

Mensajes Generados
     driver_id  score_WAPU  score_WADO  score_PACE PU Succeded  \
0       427199    0.225269    0.023636                0.006732   
1       370425    0.090827    0.019770                0.014575   
2       653770    0.190338    0.062652                0.012735   
3       505787    0.051174    0.036216                0.035276   
4       476245    0.240611    0.029297                0.029911   
..         ...         ...         ...                     ...   
142     405141         NaN         NaN                     NaN   
143     512711         NaN         NaN                     NaN   
144     619693         NaN         NaN                     NaN   
145     650097         NaN         NaN                     NaN   
146     676819         NaN         NaN                     NaN   

     score_PACE DO Succeded  score_PACE PU vs DO  score_Online Rate  \
0                  0.045546             0.059613           0.997046   
1                  0.078823             0.0563

In [285]:
dt = datetime.datetime.now()
print("Week number of the year : ", dt.strftime("%W"))

Week number of the year :  11


In [286]:
#Guardamos el csv con los scoring y su mensaje
final_df.to_csv(f'backups/Ranking_6W{dt.strftime("%W")}.csv')
print('csv generado')

csv generado


In [287]:
#Guardamos un CSV para que veamos unicamente los mensajes enviados a cada driver en la semana actual, para poder consultarlo más adleante.
final_df['week'] =  dt.strftime("%W")
mensajes_enviados_df = final_df[['week','driver_id','mensaje']]
mensajes_enviados_df.to_csv(f'backups/Mensajes_W{dt.strftime("%W")}.csv')
print('csv generado')

csv generado


In [288]:
#Mergeamos el correo que queremos envíar de sus datos, con el mensaje generado en función de sus métricas.
df_merged = pd.merge(email_df,final_df, on='driver_id', how='inner')
df_merged['email'] = 'bryanrarogal@gmail.com'
df_merged

,driver_id,WAPU,WADO,PACE PU Succeded,PACE DO Succeded,PACE PU vs DO,Online Rate,Cancelations,IAB,Total_Score_x,...,score_PACE PU Succeded,score_PACE DO Succeded,score_PACE PU vs DO,score_Online Rate,score_Cancelations,score_IAB,Total_Score_y,mensaje,week,email
0,427199,3.609494,2.784293,3.981534,4.021644,-0.040110,0.997046,0.000000,0.936941,1.573191,...,0.006732,0.045546,0.059613,0.997046,0.000000,0.936941,1.573191,¡Excelente trabajo!,11,bryanrarogal@gmail.com
1,370425,2.280904,2.573067,4.493187,4.748133,-0.254946,0.965720,0.007538,0.859259,1.557083,...,0.014575,0.078823,0.056363,0.965720,0.007538,0.859259,1.557083,¡Excelente trabajo!,11,bryanrarogal@gmail.com
2,653770,3.264297,4.915756,4.373149,4.111342,0.261807,0.963806,0.006481,0.977778,1.555541,...,0.012735,0.049655,0.064181,0.963806,0.006481,0.977778,1.555541,¡Excelente trabajo!,11,bryanrarogal@gmail.com
3,505787,1.889048,3.471548,5.843615,5.006640,0.836975,0.989833,0.000000,0.850000,1.553621,...,0.035276,0.090664,0.072882,0.989833,0.000000,0.850000,1.553621,¡Excelente trabajo!,11,bryanrarogal@gmail.com
4,476245,3.761104,3.093548,5.493656,5.115362,0.378294,1.000000,0.002096,1.000000,1.536497,...,0.029911,0.095645,0.065943,1.000000,0.002096,1.000000,1.536497,¡Excelente trabajo!,11,bryanrarogal@gmail.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,405141,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,1.000000,NaN,...,NaN,NaN,NaN,1.000000,0.000000,1.000000,NaN,No hay puntuación total disponible.,11,bryanrarogal@gmail.com
149,512711,NaN,NaN,NaN,NaN,NaN,0.762333,1.000000,0.000000,NaN,...,NaN,NaN,NaN,0.762333,1.000000,0.000000,NaN,No hay puntuación total disponible.,11,bryanrarogal@gmail.com
150,619693,NaN,NaN,NaN,NaN,NaN,0.552333,1.000000,0.000000,NaN,...,NaN,NaN,NaN,0.552333,1.000000,0.000000,NaN,No hay puntuación total disponible.,11,bryanrarogal@gmail.com
151,650097,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.400000,NaN,...,NaN,NaN,NaN,1.000000,0.000000,0.400000,NaN,No hay puntuación total disponible.,11,bryanrarogal@gmail.com


In [289]:
#Limpiamos los datos para que el correo se vea más bonito.
df_merged['WAPU'] = round(df_merged['WAPU'],2)
df_merged['WADO'] = round(df_merged['WADO'],2)
df_merged['PACE PU Succeded'] = round(df_merged['PACE PU Succeded'],2)
df_merged['PACE DO Succeded'] = round(df_merged['PACE DO Succeded'],2)
df_merged['PACE PU vs DO'] = round(df_merged['PACE PU vs DO'],2)
df_merged['Online Rate'] = (df_merged['Online Rate'] * 100).round(2).astype(str) + '%'
df_merged['Cancelations'] = (df_merged['Cancelations'] * 100).round(2).astype(str) + '%'
df_merged['IAB'] = (df_merged['IAB'] * 100).round(2).astype(str) + '%'


In [290]:
#Este será uno de los correos de los datos que usaremos para enviar su correo.
df_merged

,driver_id,WAPU,WADO,PACE PU Succeded,PACE DO Succeded,PACE PU vs DO,Online Rate,Cancelations,IAB,Total_Score_x,...,score_PACE PU Succeded,score_PACE DO Succeded,score_PACE PU vs DO,score_Online Rate,score_Cancelations,score_IAB,Total_Score_y,mensaje,week,email
0,427199,3.61,2.78,3.98,4.02,-0.04,99.7%,0.0%,93.69%,1.573191,...,0.006732,0.045546,0.059613,0.997046,0.000000,0.936941,1.573191,¡Excelente trabajo!,11,bryanrarogal@gmail.com
1,370425,2.28,2.57,4.49,4.75,-0.25,96.57%,0.75%,85.93%,1.557083,...,0.014575,0.078823,0.056363,0.965720,0.007538,0.859259,1.557083,¡Excelente trabajo!,11,bryanrarogal@gmail.com
2,653770,3.26,4.92,4.37,4.11,0.26,96.38%,0.65%,97.78%,1.555541,...,0.012735,0.049655,0.064181,0.963806,0.006481,0.977778,1.555541,¡Excelente trabajo!,11,bryanrarogal@gmail.com
3,505787,1.89,3.47,5.84,5.01,0.84,98.98%,0.0%,85.0%,1.553621,...,0.035276,0.090664,0.072882,0.989833,0.000000,0.850000,1.553621,¡Excelente trabajo!,11,bryanrarogal@gmail.com
4,476245,3.76,3.09,5.49,5.12,0.38,100.0%,0.21%,100.0%,1.536497,...,0.029911,0.095645,0.065943,1.000000,0.002096,1.000000,1.536497,¡Excelente trabajo!,11,bryanrarogal@gmail.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,405141,NaN,NaN,NaN,NaN,NaN,100.0%,0.0%,100.0%,NaN,...,NaN,NaN,NaN,1.000000,0.000000,1.000000,NaN,No hay puntuación total disponible.,11,bryanrarogal@gmail.com
149,512711,NaN,NaN,NaN,NaN,NaN,76.23%,100.0%,0.0%,NaN,...,NaN,NaN,NaN,0.762333,1.000000,0.000000,NaN,No hay puntuación total disponible.,11,bryanrarogal@gmail.com
150,619693,NaN,NaN,NaN,NaN,NaN,55.23%,100.0%,0.0%,NaN,...,NaN,NaN,NaN,0.552333,1.000000,0.000000,NaN,No hay puntuación total disponible.,11,bryanrarogal@gmail.com
151,650097,NaN,NaN,NaN,NaN,NaN,100.0%,0.0%,40.0%,NaN,...,NaN,NaN,NaN,1.000000,0.000000,0.400000,NaN,No hay puntuación total disponible.,11,bryanrarogal@gmail.com


# Generamos los datos de la semana anterior

In [291]:
semana_anterior = dt.isocalendar()[1] - 1
df_semana_anterior = performance[performance['week'] == semana_anterior]  #performance[performance['week'] == 7]esto se usara para cuando el dataset este updated
print('df_semana_anterior_generado')

,driver_id,week,transport_type,tenure,deliveries_invoiced,e2e_invoiced,client_price_invoiced,driver_earnings_invoiced,deliveries_not_invoiced,e2e_not_invoiced,...,PACE PU Succeded,PACE DO Succeded,PACE PU vs DO,Connection Rate,Deliveries,Online,Online Rate,Del/h,Cancelations,IAB
8,144250,10.0,e_motorbikexl,NaN,2,3485,13.56,1.03,0,0,...,5.345983,4.854251,0.491732,1.000000,2,24.00,1.000000,0.083333,0.000000,1.00
17,276993,10.0,e_motorbikexl,182.0,24,52133,159.54,11.71,1,1428,...,5.901159,3.604337,2.296822,0.989167,24,23.74,0.989167,1.010952,0.043478,0.75
25,280920,10.0,e_motorbikexl,181.0,41,76146,255.90,17.15,2,628,...,4.199034,3.879126,0.319907,0.949667,41,28.49,0.949667,1.439101,0.047619,0.20
31,283943,10.0,e_motorbikexl,180.0,26,55374,168.60,13.57,3,2789,...,5.158062,3.944137,1.213924,0.983750,26,23.61,0.983750,1.101228,0.038462,1.00
33,284449,10.0,bike,180.0,6,9391,31.45,1.42,1,1845,...,5.351997,3.027322,2.324674,0.200000,6,5.00,0.200000,1.200000,0.000000,0.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1047,681650,10.0,bike,44.0,21,43498,124.66,0.00,0,0,...,7.177413,5.974967,1.202445,0.958667,21,14.38,0.958667,1.460362,0.000000,0.00
1056,683252,10.0,e_bike,43.0,34,74599,193.40,0.00,2,2171,...,7.674094,6.120999,1.553095,0.988333,34,29.65,0.988333,1.146712,0.058824,0.60
1065,685061,10.0,bike,42.0,2,1731,11.70,0.00,0,0,...,5.992166,4.340170,1.651996,1.000000,2,15.00,1.000000,0.133333,0.000000,0.00
1069,685063,10.0,bike,42.0,0,0,0.00,0.00,1,1334,...,NaN,NaN,NaN,0.997333,0,14.96,0.997333,0.000000,1.000000,0.00


In [292]:
#Cremaos un nuevo df con las columnas que nos interesan.
df_semana_anterior_trans = df_semana_anterior.loc[:, columnas]

In [293]:
ad_columns = ['driver_id', 'transport_type']
t_columns = ad_columns + columnas_numericas
# Seleccionar las columnas del DataFrame df_semana_anterior
df_semana_anterior = df_semana_anterior.loc[:, t_columns]

print("DataFrame filtrado y seleccionado con columnas adicionales:")
print(transformed_performance)


DataFrame filtrado y seleccionado con columnas adicionales:
      driver_id  week transport_type  E2E Invoiced  E2E Succeeded  \
0        144250   2.0  e_motorbikexl     29.966667      30.691667   
1        144250   3.0  e_motorbikexl     29.485417      29.485417   
2        144250   4.0  e_motorbikexl     21.078333      20.289583   
3        144250   5.0  e_motorbikexl     30.050000      29.768519   
4        144250   6.0  e_motorbikexl     23.033333      23.637037   
...         ...   ...            ...           ...            ...   
1079     694272   6.0         e_bike     19.042157      19.042157   
1080     694272   7.0         e_bike     23.339583      23.339583   
1081     694272   8.0         e_bike     19.783333      19.783333   
1082     694272   9.0         e_bike     23.279487      23.452778   
1083     694272  10.0         e_bike     23.858974      23.858974   

      PACE Succeded      WAPU      WADO  PACE PU Succeded  PACE DO Succeded  \
0          6.680634  4.743750  4

In [294]:

#Normalizamos y sacamos nuevas columnas con el scooring de las columnas que hemos seleccionado
df_semana_anterior['score_WAPU'] = (df_semana_anterior['WAPU'] - df_semana_anterior['WAPU'].min()) / (df_semana_anterior['WAPU'].max() - df_semana_anterior['WAPU'].min())
df_semana_anterior['score_WADO'] = (df_semana_anterior['WADO'] - df_semana_anterior['WADO'].min()) / (df_semana_anterior['WADO'].max() - df_semana_anterior['WADO'].min())
df_semana_anterior['score_PACE PU Succeded'] = (df_semana_anterior['PACE PU Succeded'] - df_semana_anterior['PACE PU Succeded'].min()) / (df_semana_anterior['PACE PU Succeded'].max() - df_semana_anterior['PACE PU Succeded'].min())
df_semana_anterior['score_PACE DO Succeded'] = (df_semana_anterior['PACE DO Succeded'] - df_semana_anterior['PACE DO Succeded'].min()) / (df_semana_anterior['PACE DO Succeded'].max() - df_semana_anterior['PACE DO Succeded'].min())
df_semana_anterior['score_PACE PU vs DO'] = (df_semana_anterior['PACE PU vs DO'] - df_semana_anterior['PACE PU vs DO'].min()) / (df_semana_anterior['PACE PU vs DO'].max() - df_semana_anterior['PACE PU vs DO'].min())
df_semana_anterior['score_Online Rate'] = df_semana_anterior['Online Rate']
df_semana_anterior['score_Cancelations'] = df_semana_anterior['Cancelations']
df_semana_anterior['score_IAB'] = df_semana_anterior['IAB']

#Sacamos un único scooring sumando las positivas y restando las que són negativas.
df_semana_anterior['Total_Score'] = -df_semana_anterior['score_WAPU']-df_semana_anterior['score_WADO']-df_semana_anterior['score_PACE PU Succeded']-df_semana_anterior['score_PACE DO Succeded']-df_semana_anterior['score_PACE PU vs DO']+df_semana_anterior['score_Online Rate']-df_semana_anterior['score_Cancelations']+df_semana_anterior['score_IAB']

#Generamos el ranking para ver quienes son los tops.
df_semana_anterior = df_semana_anterior.sort_values(by='Total_Score',ascending=False).reset_index(drop=True)

#generamos csv para subirlo a tableau
df_semana_anterior.to_csv('backups/rankingw-1.csv',index=True,index_label='rank')
print('csv_generado')

csv_generado


In [295]:
#creamos un df de los datos de los drivers para poder incluir sus datos en el mensaje y poder concatenar su correo electronico.
email_df1w = df_semana_anterior.sort_values(by='Total_Score',ascending=False).reset_index(drop=True)
email_df1w = email_df1w.drop(['transport_type','score_WAPU','score_WADO','score_PACE PU Succeded','score_PACE DO Succeded','score_PACE PU vs DO','score_Online Rate','score_Cancelations','score_IAB'],axis=1)

In [296]:
#Eliminamos las columnas que no necesitamos para que el mensaje sea más facíl de hacer.
final_df2 = df_semana_anterior.drop(['transport_type','WAPU','WADO','PACE PU Succeded','PACE DO Succeded','PACE PU vs DO','Online Rate','Cancelations','IAB'],axis=1)
final_df2

,driver_id,score_WAPU,score_WADO,score_PACE PU Succeded,score_PACE DO Succeded,score_PACE PU vs DO,score_Online Rate,score_Cancelations,score_IAB,Total_Score
0,370425,0.066198,0.006058,0.228341,0.274350,0.342679,0.997667,0.000000,1.0,1.080040
1,465779,0.103617,0.040794,0.243374,0.207838,0.381403,1.000000,0.000000,1.0,1.022973
2,440895,0.125400,0.032375,0.258148,0.221952,0.387957,0.996000,0.000000,1.0,0.970167
3,427199,0.106216,0.013037,0.256945,0.299773,0.356122,0.998750,0.000000,1.0,0.966657
4,653770,0.112512,0.023843,0.252011,0.295209,0.353874,0.988000,0.000000,1.0,0.950551
...,...,...,...,...,...,...,...,...,...,...
110,471437,1.000000,0.026690,0.616749,0.326867,0.641232,0.957917,0.774194,0.0,-2.427814
111,457604,NaN,NaN,NaN,NaN,NaN,0.998000,0.000000,0.0,NaN
112,676819,NaN,NaN,NaN,NaN,NaN,0.953083,1.000000,0.0,NaN
113,678593,NaN,NaN,NaN,NaN,NaN,0.463667,1.000000,0.0,NaN


In [297]:
# Calcular los valores mínimo y máximo de la columna 'score'
score_min = final_df2['Total_Score'].min()
score_max = final_df2['Total_Score'].max()

# Definir los porcentajes para los rangos(se puede cambiar, 
#...pero si queremos hacer más grupos habría que cambiar el códgio de la función)
porcentaje_superior = 0.9
porcentaje_medio = 0.7
porcentaje_bajo = 0.5

# Calcular los rangos en función de los porcentajes
rango_superior = score_min + (score_max - score_min) * porcentaje_superior
rango_medio = score_min + (score_max - score_min) * porcentaje_medio
rango_bajo = score_min + (score_max - score_min) * porcentaje_bajo
        
final_df2['mensaje'] = final_df2.apply(enviar_mensaje, axis=1)
print(f'Mensajes Generados')
print(final_df2)

Mensajes Generados
     driver_id  score_WAPU  score_WADO  score_PACE PU Succeded  \
0       370425    0.066198    0.006058                0.228341   
1       465779    0.103617    0.040794                0.243374   
2       440895    0.125400    0.032375                0.258148   
3       427199    0.106216    0.013037                0.256945   
4       653770    0.112512    0.023843                0.252011   
..         ...         ...         ...                     ...   
110     471437    1.000000    0.026690                0.616749   
111     457604         NaN         NaN                     NaN   
112     676819         NaN         NaN                     NaN   
113     678593         NaN         NaN                     NaN   
114     685063         NaN         NaN                     NaN   

     score_PACE DO Succeded  score_PACE PU vs DO  score_Online Rate  \
0                  0.274350             0.342679           0.997667   
1                  0.207838             0.3814

In [298]:
#Guardamos el csv con los scoring y su mensaje
final_df2.to_csv(f'backups/Ranking_1W{dt.strftime("%W")}.csv')
print('csv generado')

csv generado


In [299]:
#Guardamos un CSV para que veamos unicamente los mensajes enviados a cada driver en la semana actual, para poder consultarlo más adleante.
final_df2['week'] =  dt.strftime("%W")
mensajes_enviados_df2 = final_df2[['week','driver_id','mensaje']]
mensajes_enviados_df2.to_csv(f'backups/Mensajes_W{dt.strftime("%W")}.csv')
print('csv generado')

csv generado


In [300]:
#Mergeamos el correo que queremos envíar de sus datos, con el mensaje generado en función de sus métricas.
df_merged2 = pd.merge(email_df1w,final_df2, on='driver_id', how='inner')
df_merged2['email'] = 'bryanrarogal@gmail.com'
df_merged2

,driver_id,WAPU,WADO,PACE PU Succeded,PACE DO Succeded,PACE PU vs DO,Online Rate,Cancelations,IAB,Total_Score_x,...,score_PACE PU Succeded,score_PACE DO Succeded,score_PACE PU vs DO,score_Online Rate,score_Cancelations,score_IAB,Total_Score_y,mensaje,week,email
0,370425,1.963399,2.043791,4.091949,4.363111,-0.271162,0.997667,0.000000,1.0,1.080040,...,0.228341,0.274350,0.342679,0.997667,0.000000,1.0,1.080040,¡Excelente trabajo!,11,bryanrarogal@gmail.com
1,465779,2.668116,7.105072,4.300807,3.917657,0.383150,1.000000,0.000000,1.0,1.022973,...,0.243374,0.207838,0.381403,1.000000,0.000000,1.0,1.022973,¡Excelente trabajo!,11,bryanrarogal@gmail.com
2,440895,3.078369,5.878369,4.506066,4.012185,0.493881,0.996000,0.000000,1.0,0.970167,...,0.258148,0.221952,0.387957,0.996000,0.000000,1.0,0.970167,¡Excelente trabajo!,11,bryanrarogal@gmail.com
3,427199,2.717063,3.060714,4.489351,4.533381,-0.044030,0.998750,0.000000,1.0,0.966657,...,0.256945,0.299773,0.356122,0.998750,0.000000,1.0,0.966657,¡Excelente trabajo!,11,bryanrarogal@gmail.com
4,653770,2.835648,4.635185,4.420806,4.502808,-0.082002,0.988000,0.000000,1.0,0.950551,...,0.252011,0.295209,0.353874,0.988000,0.000000,1.0,0.950551,¡Excelente trabajo!,11,bryanrarogal@gmail.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,471437,19.550000,5.050000,9.488215,4.714833,4.773382,0.957917,0.774194,0.0,-2.427814,...,0.616749,0.326867,0.641232,0.957917,0.774194,0.0,-2.427814,¡Tienes que mejorar la puntualidad!,11,bryanrarogal@gmail.com
111,457604,NaN,NaN,NaN,NaN,NaN,0.998000,0.000000,0.0,NaN,...,NaN,NaN,NaN,0.998000,0.000000,0.0,NaN,No hay puntuación total disponible.,11,bryanrarogal@gmail.com
112,676819,NaN,NaN,NaN,NaN,NaN,0.953083,1.000000,0.0,NaN,...,NaN,NaN,NaN,0.953083,1.000000,0.0,NaN,No hay puntuación total disponible.,11,bryanrarogal@gmail.com
113,678593,NaN,NaN,NaN,NaN,NaN,0.463667,1.000000,0.0,NaN,...,NaN,NaN,NaN,0.463667,1.000000,0.0,NaN,No hay puntuación total disponible.,11,bryanrarogal@gmail.com


In [301]:
#Limpiamos los datos para que el correo se vea más bonito.
df_merged2['WAPU'] = round(df_merged2['WAPU'],2)
df_merged2['WADO'] = round(df_merged2['WADO'],2)
df_merged2['PACE PU Succeded'] = round(df_merged2['PACE PU Succeded'],2)
df_merged2['PACE DO Succeded'] = round(df_merged2['PACE DO Succeded'],2)
df_merged2['PACE PU vs DO'] = round(df_merged2['PACE PU vs DO'],2)
df_merged2['Online Rate'] = (df_merged2['Online Rate'] * 100).round(2).astype(str) + '%'
df_merged2['Cancelations'] = (df_merged2['Cancelations'] * 100).round(2).astype(str) + '%'
df_merged2['IAB'] = (df_merged2['IAB'] * 100).round(2).astype(str) + '%'


In [302]:
#Este será uno de los correos de los datos que usaremos para enviar su correo.
df_merged2.to_csv("dataw-1.csv")
print('csv generado')

csv generado


In [303]:
test6w = df_merged.drop(['week','mensaje'],axis=1)

In [304]:
test6w

,driver_id,WAPU,WADO,PACE PU Succeded,PACE DO Succeded,PACE PU vs DO,Online Rate,Cancelations,IAB,Total_Score_x,score_WAPU,score_WADO,score_PACE PU Succeded,score_PACE DO Succeded,score_PACE PU vs DO,score_Online Rate,score_Cancelations,score_IAB,Total_Score_y,email
0,427199,3.61,2.78,3.98,4.02,-0.04,99.7%,0.0%,93.69%,1.573191,0.225269,0.023636,0.006732,0.045546,0.059613,0.997046,0.000000,0.936941,1.573191,bryanrarogal@gmail.com
1,370425,2.28,2.57,4.49,4.75,-0.25,96.57%,0.75%,85.93%,1.557083,0.090827,0.019770,0.014575,0.078823,0.056363,0.965720,0.007538,0.859259,1.557083,bryanrarogal@gmail.com
2,653770,3.26,4.92,4.37,4.11,0.26,96.38%,0.65%,97.78%,1.555541,0.190338,0.062652,0.012735,0.049655,0.064181,0.963806,0.006481,0.977778,1.555541,bryanrarogal@gmail.com
3,505787,1.89,3.47,5.84,5.01,0.84,98.98%,0.0%,85.0%,1.553621,0.051174,0.036216,0.035276,0.090664,0.072882,0.989833,0.000000,0.850000,1.553621,bryanrarogal@gmail.com
4,476245,3.76,3.09,5.49,5.12,0.38,100.0%,0.21%,100.0%,1.536497,0.240611,0.029297,0.029911,0.095645,0.065943,1.000000,0.002096,1.000000,1.536497,bryanrarogal@gmail.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,405141,NaN,NaN,NaN,NaN,NaN,100.0%,0.0%,100.0%,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,1.000000,NaN,bryanrarogal@gmail.com
149,512711,NaN,NaN,NaN,NaN,NaN,76.23%,100.0%,0.0%,NaN,NaN,NaN,NaN,NaN,NaN,0.762333,1.000000,0.000000,NaN,bryanrarogal@gmail.com
150,619693,NaN,NaN,NaN,NaN,NaN,55.23%,100.0%,0.0%,NaN,NaN,NaN,NaN,NaN,NaN,0.552333,1.000000,0.000000,NaN,bryanrarogal@gmail.com
151,650097,NaN,NaN,NaN,NaN,NaN,100.0%,0.0%,40.0%,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.400000,NaN,bryanrarogal@gmail.com


In [305]:
df_merged2

,driver_id,WAPU,WADO,PACE PU Succeded,PACE DO Succeded,PACE PU vs DO,Online Rate,Cancelations,IAB,Total_Score_x,...,score_PACE PU Succeded,score_PACE DO Succeded,score_PACE PU vs DO,score_Online Rate,score_Cancelations,score_IAB,Total_Score_y,mensaje,week,email
0,370425,1.96,2.04,4.09,4.36,-0.27,99.77%,0.0%,100.0%,1.080040,...,0.228341,0.274350,0.342679,0.997667,0.000000,1.0,1.080040,¡Excelente trabajo!,11,bryanrarogal@gmail.com
1,465779,2.67,7.11,4.30,3.92,0.38,100.0%,0.0%,100.0%,1.022973,...,0.243374,0.207838,0.381403,1.000000,0.000000,1.0,1.022973,¡Excelente trabajo!,11,bryanrarogal@gmail.com
2,440895,3.08,5.88,4.51,4.01,0.49,99.6%,0.0%,100.0%,0.970167,...,0.258148,0.221952,0.387957,0.996000,0.000000,1.0,0.970167,¡Excelente trabajo!,11,bryanrarogal@gmail.com
3,427199,2.72,3.06,4.49,4.53,-0.04,99.87%,0.0%,100.0%,0.966657,...,0.256945,0.299773,0.356122,0.998750,0.000000,1.0,0.966657,¡Excelente trabajo!,11,bryanrarogal@gmail.com
4,653770,2.84,4.64,4.42,4.50,-0.08,98.8%,0.0%,100.0%,0.950551,...,0.252011,0.295209,0.353874,0.988000,0.000000,1.0,0.950551,¡Excelente trabajo!,11,bryanrarogal@gmail.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,471437,19.55,5.05,9.49,4.71,4.77,95.79%,77.42%,0.0%,-2.427814,...,0.616749,0.326867,0.641232,0.957917,0.774194,0.0,-2.427814,¡Tienes que mejorar la puntualidad!,11,bryanrarogal@gmail.com
111,457604,NaN,NaN,NaN,NaN,NaN,99.8%,0.0%,0.0%,NaN,...,NaN,NaN,NaN,0.998000,0.000000,0.0,NaN,No hay puntuación total disponible.,11,bryanrarogal@gmail.com
112,676819,NaN,NaN,NaN,NaN,NaN,95.31%,100.0%,0.0%,NaN,...,NaN,NaN,NaN,0.953083,1.000000,0.0,NaN,No hay puntuación total disponible.,11,bryanrarogal@gmail.com
113,678593,NaN,NaN,NaN,NaN,NaN,46.37%,100.0%,0.0%,NaN,...,NaN,NaN,NaN,0.463667,1.000000,0.0,NaN,No hay puntuación total disponible.,11,bryanrarogal@gmail.com


In [306]:
test1w = df_merged2.drop(['week'],axis=1)

# DATA FRAME FINAL PARA LOS CORREOS

In [308]:
merged_test = pd.merge(test1w,test6w, on ='driver_id',how='left')
print('dataframe final generado y listo para envío')

dataframe final generado y listo para envío


In [309]:
merged_test.columns

Index(['driver_id', 'WAPU_x', 'WADO_x', 'PACE PU Succeded_x',
       'PACE DO Succeded_x', 'PACE PU vs DO_x', 'Online Rate_x',
       'Cancelations_x', 'IAB_x', 'Total_Score_x_x', 'score_WAPU_x',
       'score_WADO_x', 'score_PACE PU Succeded_x', 'score_PACE DO Succeded_x',
       'score_PACE PU vs DO_x', 'score_Online Rate_x', 'score_Cancelations_x',
       'score_IAB_x', 'Total_Score_y_x', 'mensaje', 'email_x', 'WAPU_y',
       'WADO_y', 'PACE PU Succeded_y', 'PACE DO Succeded_y', 'PACE PU vs DO_y',
       'Online Rate_y', 'Cancelations_y', 'IAB_y', 'Total_Score_x_y',
       'score_WAPU_y', 'score_WADO_y', 'score_PACE PU Succeded_y',
       'score_PACE DO Succeded_y', 'score_PACE PU vs DO_y',
       'score_Online Rate_y', 'score_Cancelations_y', 'score_IAB_y',
       'Total_Score_y_y', 'email_y'],
      dtype='object')

In [261]:
#Cambiamos algunos correos para que los compis lo vean
merged_test.at[0,'email_y'] = dotenv_values('.env')['email_1']
merged_test.at[1,'email_y'] = dotenv_values('.env')['email_2']
merged_test.at[2,'email_y'] = dotenv_values('.env')['email_3']
merged_test.at[3,'email_y'] = dotenv_values('.env')['email_4']
merged_test.at[4,'email_y'] = dotenv_values('.env')['email_5']
merged_test.at[5,'email_y'] = dotenv_values('.env')['email_6']
merged_test.at[6,'email_y'] = dotenv_values('.env')['email_7']
merged_test.at[7,'email_y'] = dotenv_values('.env')['email_8']
merged_test.at[8,'email_y'] = dotenv_values('.env')['email_9']
merged_test.at[9,'email_y'] = dotenv_values('.env')['email_10']
merged_test.at[10,'email_y'] = dotenv_values('.env')['email_11']
merged_test.at[11,'email_y'] = dotenv_values('.env')['email_12']
merged_test.at[12,'email_y'] = dotenv_values('.env')['email_13']
merged_test.at[13,'email_y'] = dotenv_values('.env')['email_14']

In [314]:
merged_test

,driver_id,WAPU_x,WADO_x,PACE PU Succeded_x,PACE DO Succeded_x,PACE PU vs DO_x,Online Rate_x,Cancelations_x,IAB_x,Total_Score_x_x,...,score_WAPU_y,score_WADO_y,score_PACE PU Succeded_y,score_PACE DO Succeded_y,score_PACE PU vs DO_y,score_Online Rate_y,score_Cancelations_y,score_IAB_y,Total_Score_y_y,email_y
0,370425,1.96,2.04,4.09,4.36,-0.27,99.77%,0.0%,100.0%,1.080040,...,0.090827,0.019770,0.014575,0.078823,0.056363,0.965720,0.007538,0.859259,1.557083,bryanrarogal@gmail.com
1,465779,2.67,7.11,4.30,3.92,0.38,100.0%,0.0%,100.0%,1.022973,...,0.206490,0.095742,0.007547,0.033355,0.064444,0.999259,0.029925,0.950000,1.511757,bryanrarogal@gmail.com
2,440895,3.08,5.88,4.51,4.01,0.49,99.6%,0.0%,100.0%,0.970167,...,0.191036,0.072177,0.015218,0.063204,0.062156,1.010643,0.015680,0.921465,1.512637,bryanrarogal@gmail.com
3,427199,2.72,3.06,4.49,4.53,-0.04,99.87%,0.0%,100.0%,0.966657,...,0.225269,0.023636,0.006732,0.045546,0.059613,0.997046,0.000000,0.936941,1.573191,bryanrarogal@gmail.com
4,653770,2.84,4.64,4.42,4.50,-0.08,98.8%,0.0%,100.0%,0.950551,...,0.190338,0.062652,0.012735,0.049655,0.064181,0.963806,0.006481,0.977778,1.555541,bryanrarogal@gmail.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,471437,19.55,5.05,9.49,4.71,4.77,95.79%,77.42%,0.0%,-2.427814,...,0.724183,0.094503,0.059745,0.116445,0.088516,0.964547,0.344115,0.000000,-0.462960,bryanrarogal@gmail.com
117,457604,NaN,NaN,NaN,NaN,NaN,99.8%,0.0%,0.0%,NaN,...,0.099388,0.041940,0.095495,0.091522,0.132029,0.984046,0.111111,0.000000,0.412561,bryanrarogal@gmail.com
118,676819,NaN,NaN,NaN,NaN,NaN,95.31%,100.0%,0.0%,NaN,...,NaN,NaN,NaN,NaN,NaN,0.870485,0.935714,0.000000,NaN,bryanrarogal@gmail.com
119,678593,NaN,NaN,NaN,NaN,NaN,46.37%,100.0%,0.0%,NaN,...,0.166531,0.017734,0.207400,0.103058,0.238658,0.694778,0.255357,0.000000,-0.293959,bryanrarogal@gmail.com


# ENVÍO DE LOS MAILS!!!

In [311]:
email_emisor = 'jbryanrd@gmail.com'
email_contraseña = dotenv_values('.env')['PWDG']
email_receptor = 'bryanrarogal@gmail.com'

In [312]:
# Función para enviar correos electrónicos
def enviar_correo(destinatario, asunto, cuerpo):
    # Configurar el correo electrónico
    email_emisor = 'jbryanrd@gmail.com'
    email_contraseña = dotenv_values('.env')['PWDG']

    # Crear el mensaje
    mensaje = MIMEMultipart('alternative')
    mensaje['Subject'] = asunto
    mensaje['From'] = email_emisor
    mensaje['To'] = destinatario

    # Adjuntar la parte HTML al mensaje
    parte_html = MIMEText(cuerpo, 'html')
    mensaje.attach(parte_html)

    # Enviar el correo electrónico
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(email_emisor, email_contraseña)
        smtp.send_message(mensaje)

    print("Correo electrónico enviado exitosamente.")

In [313]:
%%time
for index, row in merged_test.iterrows():
    driver_id = row['driver_id']
    destinatario = row['email_x']
    mensaje = row['mensaje']

    # Obtenemos los datos de la semana actual
    semana_actual_data = {
        'Minutos en el punto de recogida': row['WAPU_x'],
        'Minutos en el punto de entrega': row['WADO_x'],
        'Ritmo de recogida (m/km)': row['PACE PU Succeded_x'],
        'Ritmo de entrega (m/km)': row['PACE DO Succeded_x'],
        'Diferencia entre tus ritmos': row['PACE PU vs DO_x'],
        'Porcentaje de conexión': row['Online Rate_x'],
        'Porcentaje de pedidos cancelados': row['Cancelations_x'],
        'Porcentaje de puntualidad': row['IAB_x']
    }

    # Obtenemos los datos de la semana anterior
    semana_anterior_data = {
        'Minutos en el punto de recogida': row['WAPU_y'],
        'Minutos en el punto de entrega': row['WADO_y'],
        'Ritmo de recogida (m/km)': row['PACE PU Succeded_y'],
        'Ritmo de entrega (m/km)': row['PACE DO Succeded_y'],
        'Diferencia entre tus ritmos': row['PACE PU vs DO_y'],
        'Porcentaje de conexión': row['Online Rate_y'],
        'Porcentaje de pedidos cancelados': row['Cancelations_y'],
        'Porcentaje de puntualidad': row['IAB_y']
    }
    
    # Creamos un DataFrame para las métricas de ambas semanas
    df_metrics = pd.DataFrame({
        'Métrica': list(semana_actual_data.keys()),
        'Semana Actual': list(semana_actual_data.values()),
        'Semana Anterior': list(semana_anterior_data.values())
    })
    
    # Cuerpo del correo electrónico en HTML
    cuerpo_correo_html = f"""
    <html>
    <head>
      <style>
        body {{
          font-family: Arial, sans-serif;
          color: #333;
        }}
        .titulo {{
          font-size: 18px;
          font-weight: bold;
          margin-bottom: 5px;
        }}
        .valor {{
          font-size: 16px;
          color: #666;
        }}
        .mensaje {{
          margin-top: 15px;
          font-style: italic;
        }}
        table {{
          border-collapse: collapse;
          width: 100%;
        }}
        th, td {{
          padding: 8px;
          text-align: left;
          border-bottom: 1px solid #ddd;
        }}
        th {{
          background-color: #f2f2f2;
        }}
      </style>
    </head>
    <body>
      <p>👋Hola👋,</p>
      <p>🚀Te enviamos tus métricas🚀:</p>
     
      <table>
        <tr>
          <th>Métrica</th>
          <th>Semana Anterior</th>
          <th>Ultimas 6 semanas</th>
        </tr>
    """

    # Agregar filas de la tabla con las métricas de ambas semanas
    for _, metric_row in df_metrics.iterrows():
        cuerpo_correo_html += f"""
        <tr>
          <td>{metric_row['Métrica']}</td>
          <td>{metric_row['Semana Actual']}</td>
          <td>{metric_row['Semana Anterior']}</td>
        </tr>
        """

    # Cerrar el cuerpo del correo electrónico HTML
    cuerpo_correo_html += f"""
      </table>
      <p class="mensaje" style="font-size: 24px;">{mensaje} </p>
      <p>¡Gracias!</p>
    </body>
    </html>
    """

    # Enviar correo electrónico
    enviar_correo(destinatario, 'Aquí tienes tus métricas', cuerpo_correo_html)

print("Todos los correos electrónicos enviados exitosamente.")


Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.


KeyboardInterrupt: 